In [5]:
import json
from pathlib import Path
from math import radians, sin, cos, asin, sqrt

# Haversine distance in meters
R_EARTH_M = 6371000.0

def haversine_m(a, b):
    lat1, lon1 = a
    lat2, lon2 = b
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    lat1r = radians(lat1)
    lat2r = radians(lat2)
    h = sin(dlat/2)**2 + cos(lat1r) * cos(lat2r) * sin(dlon/2)**2
    return 2 * R_EARTH_M * asin(sqrt(h))

# Target location: 39.9519° N, 75.1558° W
lat0, lon0 = 39.9519, -75.1558

# Resolve path relative to typical run locations
candidates = [
    Path('waypoint_data.json'),
    Path('backend/waypoint_data.json'),
    Path.cwd() / 'waypoint_data.json',
    Path.cwd() / 'backend' / 'waypoint_data.json',
    Path.cwd().parent / 'backend' / 'waypoint_data.json',
]
wp_path = None
for p in candidates:
    if p.exists():
        wp_path = p
        break
assert wp_path is not None, f"waypoint_data.json not found; tried: {candidates}"

# Load waypoints
with open(wp_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flatten into a single list
waypoints = []
if isinstance(data, list):
    waypoints = data
elif isinstance(data, dict):
    # Prefer common keys if present
    if 'waypoints' in data and isinstance(data['waypoints'], list):
        waypoints = data['waypoints']
    else:
        for k, v in data.items():
            if isinstance(v, list):
                waypoints.extend(v)

# Filter "markets" broadly: shop in ['marketplace','supermarket','grocery'] or name containing "market"
market_like = {'marketplace', 'supermarket', 'grocery'}

def is_market(wp: dict) -> bool:
    name = str(wp.get('name', '') or '').lower()
    shop = str(wp.get('shop', '') or '').lower()
    tags = wp.get('tags') or {}
    tag_shop = str(tags.get('shop', '') or '').lower()
    if shop in market_like or tag_shop in market_like:
        return True
    return 'market' in name

# Compute distances and collect within 500m
results = []
for wp in waypoints:
    if not is_market(wp):
        continue
    lat, lon = wp['coordinates']  # [lat, lon]
    d = haversine_m((lat0, lon0), (lat, lon))
    if d <= 500:
        results.append({
            'id': wp.get('id'),
            'name': wp.get('name'),
            'shop': wp.get('shop'),
            'amenity': wp.get('amenity'),
            'distance_m': round(d, 1),
            'lat': lat,
            'lon': lon,
        })

# Sort by distance
results.sort(key=lambda x: x['distance_m'])

print(f"Using waypoint file: {wp_path}")
print(f"Found {len(results)} market-like waypoints within 500m of ({lat0}, {lon0})")
for r in results[:10]:
    print(r)

Using waypoint file: waypoint_data.json
Found 7 market-like waypoints within 500m of (39.9519, -75.1558)
{'id': "store_('node', 11054789533)", 'name': 'Giant', 'shop': 'supermarket', 'amenity': 'nan', 'distance_m': 177.4, 'lat': 39.9514455, 'lon': -75.1538054}
{'id': "store_('node', 6556392903)", 'name': 'No. 1 Supermarket', 'shop': 'supermarket', 'amenity': 'nan', 'distance_m': 193.7, 'lat': 39.9536402, 'lon': -75.155894}
{'id': "store_('node', 9593113373)", 'name': "MOM's Organic Market", 'shop': 'supermarket', 'amenity': 'nan', 'distance_m': 267.4, 'lat': 39.9509231, 'lon': -75.1586671}
{'id': "store_('node', 6558075400)", 'name': 'Chestnut Food Market', 'shop': 'supermarket', 'amenity': 'nan', 'distance_m': 289.7, 'lat': 39.9495163, 'lon': -75.1544281}
{'id': "store_('node', 6556417046)", 'name': 'New World Laundromat & Food Market', 'shop': 'supermarket', 'amenity': 'nan', 'distance_m': 314.8, 'lat': 39.9547139, 'lon': -75.156204}
{'id': "store_('node', 1702834399)", 'name': 'Asia